In [ ]:
import os

# os.environ['KMP_DUPLICATE_LIB_OK']='True'

import numpy as np
import numpy.ma as ma
import os
import time

import pandas as pd
# from lmfit.models import GaussianModel, Model
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn import mixture, metrics

# from shapely.geometry import Polygon

from scipy.stats.distributions import norm
from sklearn.neighbors import KernelDensity
from scipy.stats import gaussian_kde

from matplotlib import pyplot as plt
from matplotlib import cm

from skimage.measure import EllipseModel
from matplotlib.patches import Ellipse

from scipy import ndimage as ndi
from scipy.spatial import distance
from numpy import linalg

import skimage
from skimage import io, transform
from skimage.draw import ellipse
from skimage.color import rgb2gray
from skimage import filters
from skimage.morphology import disk
from skimage import color
from skimage import filters, util
from skimage.morphology import disk, skeletonize, ball

from PIL import Image, ImageDraw, ImageFilter, ImageOps
import copy
import cv2
import os
import sys

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.ndimage import distance_transform_edt as edt
from skimage.draw import ellipse
from skimage.measure import find_contours, approximate_polygon, subdivide_polygon
import logging
import glob

# from radio_beam.commonbeam import getMinVolEllipse
#
# from src.utils import grainPreprocess, grainShow, grainMark, grainDraw, grainApprox, grainStats, grainMorphology,grainGenerate
# from src.cfg import CfgAnglesNames, CfgBeamsNames, CfgDataset
#
# import plotly.graph_objects as go

from matplotlib.image import NonUniformImage
import matplotlib

from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl

from tqdm.notebook import tqdm
import json

from sklearn.metrics import pairwise_distances
import seaborn as sns
from matplotlib.colors import LogNorm, Normalize
import scipy
from sklearn import linear_model


In [ ]:
{'path':paths[i],
 'name':name,
 'type':types_dict[names[i]],
 'legend':[{'a_beams':legend1,'b_beams':legend2}],
 'density_curve_scatter':[{'a_beams':(x1.flatten(),y1.flatten()),'b_beams':(x2.flatten(),y2.flatten())}],
 'linear_approx_plot':[{'a_beams':(x_pred1.flatten(), y_pred1.flatten()),'b_beams':(x_pred2.flatten(), y_pred2.flatten())}],
 'linear_approx_data':[{'a_beams':{'k':k1,'b':b1,'angle':angle1,'score':score1},'b_beams':{'k':k2,'b':b2,'angle':angle2,'score':score2}}],
 'beams_length_series':[{'a_beams':all_a_beams,'b_beams':all_b_beams}],
 'pixel2meter':pixel,
 }

In [ ]:
step=2
data = open(f'bc_left_(1536,1536,1)_preprocess_step_{step}_beams.json',encoding='utf-8')
data = json.load(data)

## Раcпределение длин полуосей

In [ ]:
def ellipses_beams(data,save_name, step, N, M, indices=None,save=False, scatter_size=60,font_size=20):
    #
    # вывод распределения длин а- и б- полуосей для разных образцов
    #
    alloys_indices=range(len(data))

    if indices is not None:
        alloys_indices=indices

    matplotlib.rc('xtick', labelsize=font_size)
    matplotlib.rc('ytick', labelsize=font_size)

    markers = ['v', 's', 'D', 'd', 'p', '*','P']
    colors = ['orange', 'red','blue','green', 'indigo']

    # ax1.set_ylabel('ln( p(x) )')
    # ax2.set_ylabel('ln( p(x) )')


    beam_types=['a_beams','b_beams']
    beam_xlabel=['a, мкм','b, мкм']

    legend=[]
    for i in alloys_indices:
        legend.append(data['data'][i]['name']+' '+data['data'][i]['type'])


    for k,beam_type in enumerate(beam_types):

        fig=plt.figure(figsize=(N, M),facecolor='White')
        plt.rcParams['axes.facecolor'] = 'White'


        plt.xlabel(beam_xlabel[k],fontsize=font_size)

        # linear approx
        for i in alloys_indices:

            plt.plot(data['data'][i]['linear_approx_plot'][0][beam_type][0],
                     data['data'][i]['linear_approx_plot'][0][beam_type][1],color=colors[i])

        # density curve
        for i in alloys_indices:
            marker = markers[i]
            plt.scatter(data['data'][i]['density_curve_scatter'][0][beam_type][0],data['data'][i]['density_curve_scatter'][0][beam_type][1],  marker=marker,color=colors[i],s=scatter_size)

        if save:
            plt.savefig(f'bimodal_beams_{beam_type}_{save_name}_step_{step}.png',bbox_inches='tight')
        plt.show()

In [ ]:
N = 7
M = 7

step = 2
# save=True
save=False

save_name='original'


ellipses_beams(data,save_name, step, N, M, indices=[3,0,2],save=save)
# diametr_approx_all(path,save_name, step, N, M, indices=[0, 2, 3],save=save)

## Heatmap для двумерного распределения полуосей

In [ ]:
def beams_heatmap(data,step,saved_names,indices=None,bin_max=30,N=7,M=7,font_size=20,save=False,scatter_size=60):

    markers = ['v', 's', 'D', 'd', 'p', '*','P']
    colors = ['orange', 'red','blue','green', 'indigo']

    if indices is not None:
        alloys_indices=indices
    else:
        alloys_indices=range(len(data['data']))

    names=[]
    matrices=[]
    ellipses_num=[]
    polynom_x=[]
    polynom_y=[]

    for i in alloys_indices:
        a_beams = np.array(data['data'][i]['beams_length_series'][0]['a_beams'])
        b_beams= np.array(data['data'][i]['beams_length_series'][0]['b_beams'])

        ellipses_num.append(len(a_beams))

        matrix=np.zeros((bin_max,bin_max))

        for k,a_i in enumerate(a_beams):
            b_i=b_beams[k]
            if a_i<bin_max and b_i < bin_max:
                matrix[a_i,b_i]+=1

        matrix/=matrix.sum()
        matrices.append(matrix)
        names.append(data['data'][i]['name']+' '+data['data'][i]['type'])

    for i,matrix in enumerate(matrices):

        fig=plt.figure(figsize=(N, M),facecolor='White')
        plt.rcParams['axes.facecolor'] = 'White'

        matplotlib.rc('xtick', labelsize=font_size)
        matplotlib.rc('ytick', labelsize=font_size)

        im=plt.imshow(matrix,norm=mpl.colors.LogNorm(), cmap=plt.cm.RdYlBu_r)
        # im=ax.pcolormesh(matrices[i],norm=normalizer, cmap=plt.cm.RdYlBu_r)

        y=np.array(range(bin_max))
        x=[]
        for y_i in y:
            max_val=matrix[y_i].max()
            x.append(np.where(matrix[y_i]==max_val)[0][0])
        x=np.array(x)

        coefs = np.polyfit(x.flatten(), y.flatten(), 2)

        x=np.array(range(bin_max))
        plt.plot(x,np.polyval(coefs, x),color='green')

        for j,coef in enumerate(coefs):
            coefs[j]=round(coef,3)

        polynom_x.append(x)
        polynom_y.append(np.polyval(coefs, x).astype(np.int32))



        print(f'{names[i]}\ncoefs={coefs}\nellipses num={ellipses_num[i]}')

        fig.colorbar(im,  orientation='vertical')

        plt.xlabel('b, мкм',fontsize=font_size)
        plt.ylabel('a, мкм',fontsize=font_size)
        plt.title(names[i],fontsize=15)

        plt.xlim(0,bin_max)
        plt.ylim(0,bin_max)
        plt.grid(False)

        if save:
            # plt.savefig(f'{names[i]}_step_{step}.png',bbox_inches='tight')
            name=data['data'][i]['name']
            plt.savefig(f'{saved_names[name]}.png',bbox_inches='tight')
        plt.show()


    fig=plt.figure(figsize=(N, M),facecolor='White')
    plt.rcParams['axes.facecolor'] = 'White'

    matplotlib.rc('xtick', labelsize=font_size)
    matplotlib.rc('ytick', labelsize=font_size)


    for k,i in enumerate(alloys_indices):
        x=polynom_x[k]
        y=polynom_y[k].astype(np.int32)

        y_clean=[]
        for y_i in y:
            if y_i<0:
                y_clean.append(0)
            if y_i<bin_max and y_i>=0:
                y_clean.append(y_i)

        x_clean=x[:len(y_clean)]
        y_clean=np.array(y_clean)

        polynom_union_image=matrices[k][y_clean,x_clean]
        plt.xticks(range(0,35,10))

        plt.scatter(y_clean,polynom_union_image,marker=markers[i],color=colors[i],s=scatter_size)

    matplotlib.rc('xtick', labelsize=font_size)
    matplotlib.rc('ytick', labelsize=font_size)

    plt.xlabel('a, мкм', fontsize=font_size)
    plt.ylabel('p(a,b)', fontsize=font_size,labelpad=15)


    if save:

        plt.savefig(f'polynom_heatmap_step={step}.png',bbox_inches='tight')

    plt.show()

In [ ]:
N = 7
M = 7

step = 2
save=True
# save=False

saved_image_name='original'

names_new={'Ultra_Co11':'AB_Co11_medium',
           'Ultra_Co15':'AB_Co15_medium_small',
           'Ultra_Co25':'AB_Co25_small',
           'Ultra_Co6_2':'AB_Co6_large',
           'Ultra_Co8':'AB_Co_8_medium_small'}



beams_heatmap(data,saved_names=names_new,step=step,indices=[3,0,2], N=N, M=M,save=save, font_size=45, scatter_size=110)
